#### Planteamiento del Problema

Se desea predecir la cantidad de puntos que pueda hacer un jugador en un partido, 
a partir de estadísticas personales (del jugador) y colectivas (del equipo en donde juega). 

Imports

In [1]:
#Se importan todas las dependencias necesarias
import pandas as pd
import sqlalchemy
import psycopg2
import sqlite3
import csv
import os
import numpy as np
import scipy.stats
from scipy.stats import skew, kurtosis
from sqlalchemy import create_engine
from dotenv import load_dotenv
from nba_api.stats.endpoints import playergamelogs, teamgamelogs, teamgamelog, playerdashboardbylastngames

Obtenemos las estadísticas de los jugadores por partidos en las temporadas 2020-21, 2021-22 y 2022-23

In [2]:
season0_players = playergamelogs.PlayerGameLogs(season_nullable='2020-21') #Consulta para temporada 2020-21
season1_players = playergamelogs.PlayerGameLogs(season_nullable='2021-22') #Consulta para temporada 2021-22
season2_players = playergamelogs.PlayerGameLogs(season_nullable='2022-23') #Consulta para temporada 2022-23
players = pd.concat([season0_players.get_data_frames()[0],season1_players.get_data_frames()[0],season2_players.get_data_frames()[0]], ignore_index=True) #Dataframe de estadísticas de jugadores por partido en las tres temporadas
players.head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG
0,2020-21,1626169,Stanley Johnson,Stanley,1610612761,TOR,Toronto Raptors,0022001079,2021-05-16T00:00:00,TOR vs. IND,...,15321,4919,795,1773,17520,2490,1849,143,1612,1
1,2020-21,1629052,Oshae Brissett,Oshae,1610612754,IND,Indiana Pacers,0022001079,2021-05-16T00:00:00,IND @ TOR,...,1,16132,795,560,3883,1231,1,143,808,1
2,2020-21,203083,Andre Drummond,Andre,1610612747,LAL,Los Angeles Lakers,0022001072,2021-05-16T00:00:00,LAL @ NOP,...,1,21997,2281,7127,11703,2950,1,143,3790,1
3,2020-21,202685,Jonas Valanciunas,Jonas,1610612763,MEM,Memphis Grizzlies,0022001070,2021-05-16T00:00:00,MEM @ GSW,...,15321,10833,2281,787,20143,774,1,143,898,1
4,2020-21,1628981,Bruno Fernando,Bruno,1610612737,ATL,Atlanta Hawks,0022001066,2021-05-16T00:00:00,ATL vs. HOU,...,20949,4919,6289,9732,3883,6572,1849,143,8016,1


Obtenemos las estadísticas de los equipos por partidos en las temporadas 2020-21, 2021-22 y 2022-23

In [3]:
season0_teams = teamgamelogs.TeamGameLogs(season_nullable='2020-21') #Consulta para temporada 2020-21
season1_teams = teamgamelogs.TeamGameLogs(season_nullable='2021-22') #Consulta para temporada 2021-22
season2_teams = teamgamelogs.TeamGameLogs(season_nullable='2022-23') #Consulta para temporada 2022-23
teams = pd.concat([season0_teams.get_data_frames()[0],season1_teams.get_data_frames()[0],season2_teams.get_data_frames()[0]], ignore_index=True) #Dataframe de estadísticas de equipos por partido en las tres temporadas
teams.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG
0,2020-21,1610612766,CHA,Charlotte Hornets,0022001080,2021-05-16T00:00:00,CHA @ WAS,L,48.0,37,...,938,826,751,801,332,247,786,1180,1305,1
1,2020-21,1610612749,MIL,Milwaukee Bucks,0022001068,2021-05-16T00:00:00,MIL @ CHI,L,48.0,38,...,2131,641,66,801,1868,376,786,1031,1373,1
2,2020-21,1610612746,LAC,LA Clippers,0022001074,2021-05-16T00:00:00,LAC @ OKC,L,48.0,45,...,2014,2,751,1499,2140,154,2113,1031,1305,1
3,2020-21,1610612743,DEN,Denver Nuggets,0022001076,2021-05-16T00:00:00,DEN @ POR,L,48.0,44,...,1741,14,751,1830,1361,1160,1609,762,1848,1
4,2020-21,1610612738,BOS,Boston Celtics,0022001073,2021-05-16T00:00:00,BOS @ NYK,L,48.0,35,...,1857,641,1867,490,1361,942,1437,2049,1238,1


Agregamos la columna PTSR (Puntos recibidos) por el equipo en cada partido, temporadas 2020-21, 2021-22 y 2022-23. 

Sería restar los puntos realizados menos la diferencia final de puntaje respecto al otro equipo una vez finalizado el partido.

In [4]:
teams['PTSR'] = (teams['PTS'] - teams['PLUS_MINUS']).astype(int)
teams.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG,PTSR
0,2020-21,1610612766,CHA,Charlotte Hornets,0022001080,2021-05-16T00:00:00,CHA @ WAS,L,48.0,37,...,826,751,801,332,247,786,1180,1305,1,115
1,2020-21,1610612749,MIL,Milwaukee Bucks,0022001068,2021-05-16T00:00:00,MIL @ CHI,L,48.0,38,...,641,66,801,1868,376,786,1031,1373,1,118
2,2020-21,1610612746,LAC,LA Clippers,0022001074,2021-05-16T00:00:00,LAC @ OKC,L,48.0,45,...,2,751,1499,2140,154,2113,1031,1305,1,117
3,2020-21,1610612743,DEN,Denver Nuggets,0022001076,2021-05-16T00:00:00,DEN @ POR,L,48.0,44,...,14,751,1830,1361,1160,1609,762,1848,1,132
4,2020-21,1610612738,BOS,Boston Celtics,0022001073,2021-05-16T00:00:00,BOS @ NYK,L,48.0,35,...,641,1867,490,1361,942,1437,2049,1238,1,96


Hacemos una nueva consulta a la API con otro endpoint para poder obtener el W_PCT o % de Victorias del equipo

In [5]:
teams_list = teams.TEAM_ID.unique().tolist() #El for será para encontrar el W_PCT de todos los equipos para cada partido de la temporada
teamLog = pd.DataFrame() #inicializamos para que no quede basura

for team in teams_list:
    teamLog0 = teamgamelog.TeamGameLog(season='2020-21',team_id=team) #Consulta para temporada 2020-21
    teamLog1 = teamgamelog.TeamGameLog(season='2021-22',team_id=team) #Consulta para temporada 2021-22
    teamLog2 = teamgamelog.TeamGameLog(season='2022-23',team_id=team) #Consulta para temporada 2022-23
    teamLog3 = pd.concat([teamLog0.get_data_frames()[0], teamLog1.get_data_frames()[0], teamLog2.get_data_frames()[0]],ignore_index=True) #En cada ciclo se forma el dataframe por equipo por partido en las tres temporadas 
    teamLog  = pd.concat([teamLog, teamLog3]) #Se van consolidando en un dataframe las consultas para todos los equipos

In [6]:
teamLog.head()

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612766,0022001080,"MAY 16, 2021",CHA @ WAS,L,33,39,0.458,240,37,...,0.917,10,32,42,25,8,5,11,15,110
1,1610612766,0022001064,"MAY 15, 2021",CHA @ NYK,L,33,38,0.465,265,39,...,0.750,15,36,51,28,5,2,8,18,109
2,1610612766,0022001047,"MAY 13, 2021",CHA vs. LAC,L,33,37,0.471,240,33,...,0.789,8,27,35,24,9,1,10,15,90
3,1610612766,0022000448,"MAY 11, 2021",CHA vs. DEN,L,33,36,0.478,240,42,...,0.850,15,34,49,30,9,5,17,17,112
4,1610612766,0022001020,"MAY 09, 2021",CHA vs. NOP,L,33,35,0.485,240,42,...,0.722,11,35,46,22,6,8,17,19,110


In [7]:
#Se renombran las columnas de con el mismo titulo que en el dataframe de teams, para poder hacer el merge deseado (filtrando por las columnas GAME_ID y TEAM_ID)
teamLog = teamLog.rename(columns={'Game_ID': 'GAME_ID'})
teamLog = teamLog.rename(columns={'Team_ID': 'TEAM_ID'})

In [8]:
#Se hace el merge deseado de W_PCT (% Victorias) filtrando por las columnas GAME_ID y TEAM_ID
merged_df = pd.merge(teams,teamLog, on=['GAME_ID','TEAM_ID'], how='left')
teams['W_PCT'] = merged_df['W_PCT']
teams.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG,PTSR,W_PCT
0,2020-21,1610612766,CHA,Charlotte Hornets,0022001080,2021-05-16T00:00:00,CHA @ WAS,L,48.0,37,...,751,801,332,247,786,1180,1305,1,115,0.458
1,2020-21,1610612749,MIL,Milwaukee Bucks,0022001068,2021-05-16T00:00:00,MIL @ CHI,L,48.0,38,...,66,801,1868,376,786,1031,1373,1,118,0.639
2,2020-21,1610612746,LAC,LA Clippers,0022001074,2021-05-16T00:00:00,LAC @ OKC,L,48.0,45,...,751,1499,2140,154,2113,1031,1305,1,117,0.653
3,2020-21,1610612743,DEN,Denver Nuggets,0022001076,2021-05-16T00:00:00,DEN @ POR,L,48.0,44,...,751,1830,1361,1160,1609,762,1848,1,132,0.653
4,2020-21,1610612738,BOS,Boston Celtics,0022001073,2021-05-16T00:00:00,BOS @ NYK,L,48.0,35,...,1867,490,1361,942,1437,2049,1238,1,96,0.500


In [9]:
#Para comprobar que no hayan quedado valores que no hayan hecho math entre columna GAME_ID y TEAM_ID
#Si fuese el caso, se hubiese rellenado con NaN
teams.W_PCT.isna().any()

False

Obtenemos un solo DataFrame con las columnas que necesitamos, se va a consolidar todo en el Dataframe de estadísticas de los jugadores "players"

In [10]:
#Del Dataframe "teams" se obtienen las columnas de Puntos recibidos por el equipo 'PTSR', 'PTS' Puntos realizados por el equipo y W_PCT (%Victorias)
#Se hace el merge deseado de Puntos recibidos por el equipo 'PTSR', 'PTS' Puntos realizados por el equipo y W_PCT (%Victorias); filtrando por las columnas GAME_ID y TEAM_ID
teams = teams.rename(columns={'PTS': 'PTSTeam'}) #Porque en el Dataframe players tambien hay una columna PTS

merged_df2 = pd.merge(players,teams, on=['GAME_ID','TEAM_ID'], how='left')

players[['PTSRTeam','PTSTeam','W_PCTTeam']] = merged_df2[['PTSR','PTSTeam','W_PCT']] #Se crean las nuevas columnas en el Dataframe players de acuerdo al merge realizado.

players.head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG,PTSRTeam,PTSTeam,W_PCTTeam
0,2020-21,1626169,Stanley Johnson,Stanley,1610612761,TOR,Toronto Raptors,0022001079,2021-05-16T00:00:00,TOR vs. IND,...,1773,17520,2490,1849,143,1612,1,125,113,0.375
1,2020-21,1629052,Oshae Brissett,Oshae,1610612754,IND,Indiana Pacers,0022001079,2021-05-16T00:00:00,IND @ TOR,...,560,3883,1231,1,143,808,1,113,125,0.472
2,2020-21,203083,Andre Drummond,Andre,1610612747,LAL,Los Angeles Lakers,0022001072,2021-05-16T00:00:00,LAL @ NOP,...,7127,11703,2950,1,143,3790,1,98,110,0.583
3,2020-21,202685,Jonas Valanciunas,Jonas,1610612763,MEM,Memphis Grizzlies,0022001070,2021-05-16T00:00:00,MEM @ GSW,...,787,20143,774,1,143,898,1,113,101,0.528
4,2020-21,1628981,Bruno Fernando,Bruno,1610612737,ATL,Atlanta Hawks,0022001066,2021-05-16T00:00:00,ATL vs. HOU,...,9732,3883,6572,1849,143,8016,1,95,124,0.569


In [11]:
#Para comprobar que no hayan quedado valores que no hayan hecho math entre columna GAME_ID y TEAM_ID
#Si fuese el caso, se hubiese rellenado con NaN
print(players.PTSRTeam.isna().any())
print(players.PTSTeam.isna().any())
print(players.W_PCTTeam.isna().any())

False
False
False


Se agrega una nueva columna para determinar si un equipo es local o visitante

In [12]:
def determinar_tipo(data):
    if '@' in data:
        return 'A'  # Si hay '@', entonces es visitante
    elif 'vs' in data:
        return 'H'  # Si hay 'vs', entonces es local
    
players['HOME_AWAY'] = players['MATCHUP'].apply(determinar_tipo) #Nueva columna Home (H), Away (A)

players.head(2)

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG,PTSRTeam,PTSTeam,W_PCTTeam,HOME_AWAY
0,2020-21,1626169,Stanley Johnson,Stanley,1610612761,TOR,Toronto Raptors,0022001079,2021-05-16T00:00:00,TOR vs. IND,...,17520,2490,1849,143,1612,1,125,113,0.375,H
1,2020-21,1629052,Oshae Brissett,Oshae,1610612754,IND,Indiana Pacers,0022001079,2021-05-16T00:00:00,IND @ TOR,...,3883,1231,1,143,808,1,113,125,0.472,A


Removemos las columnas que no se utilizarán en la generación del modelo predictivo.

In [13]:
players.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2',
       'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK',
       'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK', 'AVAILABLE_FLAG', 'PTSRTeam', 'PTSTeam',
       'W_PCTTeam', 'HOME_AWAY'],
      dtype='object')

In [14]:
#Remocion de columnas
players = players.drop(['WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK',
       'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK','NBA_FANTASY_PTS', 'DD2',
       'TD3'], axis = 1)

players.head(2)

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,BLKA,PF,PFD,PTS,PLUS_MINUS,AVAILABLE_FLAG,PTSRTeam,PTSTeam,W_PCTTeam,HOME_AWAY
0,2020-21,1626169,Stanley Johnson,Stanley,1610612761,TOR,Toronto Raptors,0022001079,2021-05-16T00:00:00,TOR vs. IND,...,1,1,6,24,-8,1,125,113,0.375,H
1,2020-21,1629052,Oshae Brissett,Oshae,1610612754,IND,Indiana Pacers,0022001079,2021-05-16T00:00:00,IND @ TOR,...,0,3,6,31,10,1,113,125,0.472,A


Tomamos el promedio de los últimos 5 partidos para las estadísticas por jugador

In [15]:
#Ordenar el Dataframe de forma ascendente en base a las columnas descritas.
players = players.sort_values(by=['SEASON_YEAR','PLAYER_ID','GAME_DATE'], ascending=[True,True,True])

In [16]:
#Hacemos reset el indice del dataframe en caso de que lo podamos necesitar en orden
players = players.reset_index(drop=True)

In [ ]:
def calcular_promedio_puntos(dataframe, columna, player_id, game_date, n_partidos):
    """
    Calcula el promedio de puntos de un jugador en los N partidos anteriores, excluyendo el partido actual.
    Parámetros:
      dataframe: El dataframe con los datos de los jugadores.
      game_date: El GAME_ID del partido actual.
      player_id: El PLAYER_ID del jugador para el que se quiere calcular el promedio.
      n_partidos: El número de partidos anteriores a considerar.
      columna: La columna que contiene los puntos del jugador.
    Devuelve:
      El promedio de puntos del jugador en los N partidos anteriores.
    """
    # Filtrar el dataframe para los partidos anteriores al actual y del jugador específico
    partidos_anteriores_jugador = dataframe[(dataframe['GAME_DATE'] < game_date) & (dataframe['PLAYER_ID'] == player_id)]
    
    # Si no hay suficientes partidos anteriores para el jugador, tomar los partidos disponibles
    n_partidos_disponibles = min(len(partidos_anteriores_jugador), n_partidos)
    
    # Si hay al menos un partido anterior, calcular el promedio de puntos
    if n_partidos_disponibles > 0:
        promedio_puntos = partidos_anteriores_jugador.tail(n_partidos_disponibles)[columna].mean()
    else:
        # Si no hay partidos anteriores disponibles, tomar el valor del partido actual
        promedio_puntos = dataframe[(dataframe['GAME_DATE'] == game_date) & (dataframe['PLAYER_ID'] == player_id)][columna].iloc[0]
    
    return promedio_puntos

#Lista de columnas para las cuales se obtendrán las estadísticas promediadas
lista_columnas_para_promedio = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'PTSRTeam',
       'PTSTeam', 'W_PCTTeam']

#Tomamos una listas de los jugadores y de las fechas de los partidos.
jugadores_ID = players.PLAYER_ID.unique().tolist()
dates = players.GAME_DATE.unique().tolist()


n_partidos = 5 #numero de partidos para sacar el promedio

#Se llama a la funcion para obtener las estadisticas promediadas de todos los jugadores
for columna in lista_columnas_para_promedio:
  for jugador in jugadores_ID:
      for date in dates:
        
        indice = players.index[(players['GAME_DATE']==date) & (players['PLAYER_ID']==jugador)]

        if indice.empty:
                # Si no se encontró un índice válido, pasar a la siguiente fecha
          continue

        promedio = calcular_promedio_puntos(players, columna, jugador, date, n_partidos)

        if promedio is None:
                # Pasar a la siguiente fecha
          continue
              
        players.loc[indice,columna+'_'+'PROM'] = promedio

Guardado de dataframe en un archivo .csv

In [ ]:
#se guarda el dataframe en el archivo .csv llamado "data_complete_raw.csv"
players.to_csv('/workspace/ProyectoDS/data/raw/data_complete_raw.csv', index=False)

Enviamos los registros a una DB.

In [17]:
players = pd.read_csv('/workspace/ProyectoDS/data/raw/data_complete_raw.csv')

# Se crea una base de datos llamada "NBA.db" que contendrá los datos del archivo .csv guardado previamente
con = sqlite3.connect("NBA.db")
con.execute("DROP TABLE IF EXISTS ESTADISTICAS;")
players.to_sql('ESTADISTICAS',con, index=False)

con.close()

In [20]:
con = sqlite3.connect("NBA.db")
cur = con.cursor()

# -- Obtenemos los 10 máximos anotadores.
cur.execute("""SELECT SEASON_YEAR, PLAYER_NAME, TEAM_NAME,
            PTS, FGM, FGA, FG_PCT,FG3M, FG3A, FG3_PCT
            FROM ESTADISTICAS
            ORDER BY PTS DESC
            LIMIT 10
            ;

            """)

res = cur.fetchall()

for row in res:
    print(f"Temporada: {row[0]}")
    print(f"Nombre: {row[1]}")
    print(f"Equipo: {row[2]}")
    print(f"Puntos: {row[3]}")
    print(f"Tiros convertidos: {row[4]}/{row[5]}")
    print(f"Porcentaje de aciertos: {row[6]}")
    print(f"Tiros de 3: {row[7]}/{row[8]}")
    print(f"Porcentaje tiros de 3: {row[9]}\n\n")

Temporada: 2022-23
Nombre: Damian Lillard
Equipo: Portland Trail Blazers
Puntos: 71
Tiros convertidos: 22/38
Porcentaje de aciertos: 0.579
Tiros de 3: 13/22
Porcentaje tiros de 3: 0.591


Temporada: 2022-23
Nombre: Donovan Mitchell
Equipo: Cleveland Cavaliers
Puntos: 71
Tiros convertidos: 22/34
Porcentaje de aciertos: 0.647
Tiros de 3: 7/15
Porcentaje tiros de 3: 0.467


Temporada: 2020-21
Nombre: Stephen Curry
Equipo: Golden State Warriors
Puntos: 62
Tiros convertidos: 18/31
Porcentaje de aciertos: 0.581
Tiros de 3: 8/16
Porcentaje tiros de 3: 0.5


Temporada: 2020-21
Nombre: Bradley Beal
Equipo: Washington Wizards
Puntos: 60
Tiros convertidos: 20/35
Porcentaje de aciertos: 0.571
Tiros de 3: 7/10
Porcentaje tiros de 3: 0.7


Temporada: 2020-21
Nombre: Jayson Tatum
Equipo: Boston Celtics
Puntos: 60
Tiros convertidos: 20/37
Porcentaje de aciertos: 0.541
Tiros de 3: 5/7
Porcentaje tiros de 3: 0.714


Temporada: 2021-22
Nombre: Kyrie Irving
Equipo: Brooklyn Nets
Puntos: 60
Tiros convertid

In [21]:
# -- Obtenemos los puntos de jugador por temporada (máximos anotadores en una temporada).
cur.execute("""SELECT PLAYER_NAME, TEAM_NAME, SEASON_YEAR,
              SUM(PTS) AS PTS_TOT
              FROM ESTADISTICAS
              GROUP BY PLAYER_NAME, TEAM_NAME, SEASON_YEAR
              ORDER BY PTS_TOT DESC
              LIMIT 10""")

res = cur.fetchall()

for row in res:
        print(f"Nombre: {row[0]}")
        print(f"Equipo: {row[1]}")
        print(f"Temporada: {row[2]}")
        print(f"Puntos totales: {row[3]}\n\n")

con.close()

Nombre: Jayson Tatum
Equipo: Boston Celtics
Temporada: 2022-23
Puntos totales: 2225


Nombre: Joel Embiid
Equipo: Philadelphia 76ers
Temporada: 2022-23
Puntos totales: 2183


Nombre: Trae Young
Equipo: Atlanta Hawks
Temporada: 2021-22
Puntos totales: 2155


Nombre: Luka Doncic
Equipo: Dallas Mavericks
Temporada: 2022-23
Puntos totales: 2138


Nombre: Shai Gilgeous-Alexander
Equipo: Oklahoma City Thunder
Temporada: 2022-23
Puntos totales: 2135


Nombre: DeMar DeRozan
Equipo: Chicago Bulls
Temporada: 2021-22
Puntos totales: 2118


Nombre: Joel Embiid
Equipo: Philadelphia 76ers
Temporada: 2021-22
Puntos totales: 2079


Nombre: Jayson Tatum
Equipo: Boston Celtics
Temporada: 2021-22
Puntos totales: 2046


Nombre: Stephen Curry
Equipo: Golden State Warriors
Temporada: 2020-21
Puntos totales: 2015


Nombre: Nikola Jokic
Equipo: Denver Nuggets
Temporada: 2021-22
Puntos totales: 2004




Analizamos Desscriptivo Variables Numéricas

In [22]:
numeric_columns = []
for column_name in players:
    if players[column_name].dtype == 'int64' or players[column_name].dtype == 'float64':
        numeric_columns.append(column_name)


Se muestra la media, mínimo, máximo, rango, desviación estándar, cuartiles, asimetría, curtosis y moda.

In [24]:
for column_name in numeric_columns:
    mean = players[column_name].mean()
    min = players[column_name].min()
    max = players[column_name].max()
    moda = players[column_name].mode()
    rang = max - min 
    std = players[column_name].std()
    skewness = skew(players[column_name])
    cuartil_25 = np.percentile(players[column_name],25)
    cuartil_50 = np.percentile(players[column_name],50)
    cuartil_75 = np.percentile(players[column_name],75)
    kurtosis = scipy.stats.kurtosis(players[column_name])

    print(f'Columna: {column_name}')
    print(f'    Media: {mean}')
    print(f'    Moda: {moda[0]}')
    print(f'    Mínimo: {min}')
    print(f'    Máximo: {max}')
    print(f'    Rango: {rang}')
    print(f'    Desviación estándar: {std}')
    print(f'    Cuartil 25: {cuartil_25}')
    print(f'    Cuartil 50: {cuartil_50}')
    print(f'    Cuatril 75: {cuartil_75}')
    print(f'    Kurtosis: {kurtosis}')
    print(f'    Asimetría: {skewness}')


Columna: PLAYER_ID
    Media: 1190569.214437169
    Moda: 1628969
    Mínimo: 2207
    Máximo: 1641645
    Rango: 1639438
    Desviación estándar: 660706.9301940023
    Cuartil 25: 203903.0
    Cuartil 50: 1628420.0
    Cuatril 75: 1629680.0
    Kurtosis: -1.281782639078686
    Asimetría: -0.8446181465166592
Columna: TEAM_ID
    Media: 1610612751.4898982
    Moda: 1610612743
    Mínimo: 1610612737
    Máximo: 1610612766
    Rango: 29
    Desviación estándar: 8.657299333755455
    Cuartil 25: 1610612744.0
    Cuartil 50: 1610612751.0
    Cuatril 75: 1610612759.0
    Kurtosis: -1.209389232138601
    Asimetría: -0.0010050956305153462
Columna: GAME_ID
    Media: 22104376.027271394
    Moda: 22000700
    Mínimo: 22000001
    Máximo: 22201230
    Rango: 201229
    Desviación estándar: 80734.13040925466
    Cuartil 25: 22000879.0
    Cuartil 50: 22100673.0
    Cuatril 75: 22200334.0
    Kurtosis: -1.4614406339625774
    Asimetría: -0.0692594083277923
Columna: MIN
    Media: 22.8052862496166
 

Columna: FG3_PCT_PROM
    Media: 0.2586554411653575
    Moda: 0.0
    Mínimo: 0.0
    Máximo: 1.0
    Rango: 1.0
    Desviación estándar: 0.1717737012389905
    Cuartil 25: 0.1238
    Cuartil 50: 0.275
    Cuatril 75: 0.3834
    Kurtosis: -0.3722228267527363
    Asimetría: 0.07652796597895813
Columna: FTM_PROM
    Media: 1.638770720258178
    Moda: 0.0
    Mínimo: 0.0
    Máximo: 14.4
    Rango: 14.4
    Desviación estándar: 1.7259692251474998
    Cuartil 25: 0.4
    Cuartil 50: 1.2
    Cuatril 75: 2.2
    Kurtosis: 5.293675031565655
    Asimetría: 1.991694237924623
Columna: FTA_PROM
    Media: 2.1078433595156496
    Moda: 0.0
    Mínimo: 0.0
    Máximo: 17.4
    Rango: 17.4
    Desviación estándar: 2.0918428828792144
    Cuartil 25: 0.6
    Cuartil 50: 1.6
    Cuatril 75: 2.8
    Kurtosis: 4.710227021272784
    Asimetría: 1.8699516677046686
Columna: FT_PCT_PROM
    Media: 0.42574520828054646
    Moda: 0.0
    Mínimo: 0.0
    Máximo: 1.0
    Rango: 1.0
    Desviación estándar: 0.274928

Analizamos las variables categóricas

In [84]:
categ_columns = players.select_dtypes(include=['object','category']).columns

categ_columns = categ_columns.tolist()
categ_columns.append('AVAILABLE_FLAG')
categ_columns = pd.Index(categ_columns)
categ_columns

Index(['SEASON_YEAR', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'GAME_DATE', 'MATCHUP', 'WL', 'HOME_AWAY',
       'AVAILABLE_FLAG'],
      dtype='object')